In [1]:
import numpy as np
import functools as ft
import networkx as nx

In [2]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)

In [3]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: cosine_similarity(V, x[1]))

In [4]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if cosine_similarity(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if cosine_similarity(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [7]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("g", G)
    return G

In [8]:
def checkEdge(G, A, B, threshold):
    return cosine_similarity(B, bind([G, A])) > threshold

In [9]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    for i in range(len(xs)):
        mjs = mems[:i] + mems[i + 1 :]
        mjs = bundle(mjs)

        newMems.append(bind([xs[i], bundle([G, -mjs])]))

    return newMems

In [10]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 3500, 0.047, 15

graph = nx.gnm_random_graph(NODES, EDGES)
memory = ItemMemory()

initVertices(graph, DIMENSIONS)
initNodeMem(graph, memory)
retrain(graph, memory, THRESHOLD, ITER)
G = initGraph(graph, memory)

---


In [25]:
for i in range(10):
    memsi = nodeMemoryReconstruction(
        G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
    )
    print(
        f"{n}_{i} =>",
        cosine_similarity(graph.nodes[0]["mem"], memsi[0]),
    )

29_0 => 0.2619474855578692
29_1 => 0.04194196444867849
29_2 => 0.0605971713284636
29_3 => 0.011130676381696424
29_4 => 0.07011056913709665
29_5 => 0.01637180079514616
29_6 => 0.06276692565046253
29_7 => 0.022739220681181557
29_8 => 0.058355365603473756
29_9 => 0.028874114444678998


In [24]:
mems1 = nodeMemoryReconstruction(
    G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=1
)
mems2 = nodeMemoryReconstruction(
    G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=2
)
mems5 = nodeMemoryReconstruction(
    G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=5
)
mems8 = nodeMemoryReconstruction(
    G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=8
)
for n in graph.nodes():
    print(
        f"{n}  => ",
        cosine_similarity(graph.nodes[n]["mem"], bind([G, graph.nodes[n]["hdv"]])),
    )
    print(
        f"{n}_1=> ",
        cosine_similarity(graph.nodes[n]["mem"], mems1[n]),
    )
    print(
        f"{n}_2=> ",
        cosine_similarity(graph.nodes[n]["mem"], mems2[n]),
    )
    print(
        f"{n}_5=> ",
        cosine_similarity(graph.nodes[n]["mem"], mems5[n]),
    )
    print(
        f"{n}_8=> ",
        cosine_similarity(graph.nodes[n]["mem"], mems8[n]),
    )

0  =>  0.2619474855578692
0_1=>  0.04194196444867849
0_2=>  0.0605971713284636
0_5=>  0.01637180079514616
0_8=>  0.058355365603473756
1  =>  0.2530784592193749
1_1=>  -0.037818793984801445
1_2=>  0.031606404540094425
1_5=>  -0.03645414572793314
1_8=>  0.039188096828158867
2  =>  0.24413096523303715
2_1=>  0.004276003837705734
2_2=>  0.06012376746604741
2_5=>  -0.06085553499727113
2_8=>  -0.0005952386167542705
3  =>  0.23429982146863523
3_1=>  -0.007539851229520634
3_2=>  0.04549108229652034
3_5=>  -0.02058433924276675
3_8=>  0.050152829650361774
4  =>  0.2605981663356295
4_1=>  -0.028540589549511025
4_2=>  0.08183136248397392
4_5=>  -0.03769926633484615
4_8=>  0.06750141424760224
5  =>  0.2839503941976691
5_1=>  0.053573430179427374
5_2=>  0.05392603807831377
5_5=>  0.04084658163374551
5_8=>  0.01383108924180733
6  =>  0.2511102083181977
6_1=>  0.07776630446941986
6_2=>  0.07554418468946078
6_5=>  0.025996949179857096
6_8=>  0.055297383878979034
7  =>  0.19239744142201662
7_1=>  0.0205

In [12]:
import math

SNR = 5 * math.log(DIMENSIONS / (EDGES / NODES))
print(SNR)
print(1 / SNR)

32.75540167521702
0.03052931574203858


In [13]:
query = bind([G, graph.nodes[26]["hdv"]])
res = memory.cleanup(query)
print(res[0])
print(cosine_similarity(res[1], query))

mem26
0.21219452633797514


In [14]:
# retrain(G, memory, THRESHOLD, )
# Gh = initGraph(G, memory)

In [15]:
count = 0

for n in graph.nodes():
    for m in graph.nodes():
        if n == m:
            continue
        exist = graph.has_edge(n, m)
        check = checkEdge(G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], THRESHOLD)
        if exist != check:
            count += 1
            print(n, m, exist, check)

print(count, "%.5f" % round(count / EDGES, 5))

2 16 True False
15 17 True False
16 2 True False
17 15 True False
4 0.02667
